In [1]:
import pandas as pd
import numpy as np
import hashlib
import seaborn as sb
import matplotlib.pyplot as plt

# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)





ModuleNotFoundError: No module named 'seaborn'

In [ ]:
file_id = '1RDMz-mbAwkVyLKg4gqB3DoAVowYi6VDJ' #<-- You add in here the id from you google drive file, you can find it


download = drive.CreateFile({'id': file_id})


# Download the file to a local disc
download.GetContentFile('depressionTrain.csv')  
train = pd.read_csv('depressionTrain.csv') 
train.head()

: 

In [ ]:
file_id = '1mxrhWAffzjBghqmfISz39cgKQxjUhhVP' #<-- You add in here the id from you google drive file, you can find it


download = drive.CreateFile({'id': file_id})


# Download the file to a local disc
download.GetContentFile('depressionTest.csv')  
test = pd.read_csv('depressionTest.csv') 
test.head()

: 

In [ ]:
#Description of data
train.info()


: 

In [ ]:
#Frequency Distribution
train[ "DEPRESSED"].value_counts() 


: 

In [ ]:
train_set = train.replace('Yes', 1).replace('No', 0).replace('Not married',0).replace('Married', 1)
test_set = test.replace('Yes', 1).replace('No', 0).replace('Not married',0).replace('Married', 1)


: 

In [ ]:
#Explore Gender - Depressed  relationship
gender_pivot = train_set.pivot_table(index="GENDER",values="DEPRESSED")
gender_pivot.plot.bar()

: 

In [ ]:
#Explore Education - Depression relationship
education_pivot = train_set.pivot_table(index="EDU_LEVEL",values="DEPRESSED")
education_pivot.plot.bar()

: 

In [ ]:
#Explore  numerical Age - Depressed relationship
#compare ages of those who are depressed and those who are not
depressed = train_set[train_set["DEPRESSED"] == 1]
not_depressed     = train_set[train_set["DEPRESSED"] == 0]
depressed["AGE"].plot.hist(alpha=0.5,color='blue',bins=50)
not_depressed["AGE"].plot.hist(alpha=0.5,color='red',bins=50)
plt.legend(['Depressed','Not_depressed'])
plt.show()

: 

In [ ]:
# convert age to a categorical variable
#use pandas.cut() for creating bins
cut_points = [-1,0, 5, 12, 18, 35, 60, 100]
label_names = ["Missing", 'Infant', "Child", 'Teenager', "Young Adult", 'Adult', 'Senior'] 

def process_age(df,cut_points,label_names,attribute):
    df[attribute] = df[attribute].fillna(-0.5)
    df["AGE_categories"] = pd.cut(df[attribute],cut_points,labels=label_names)
    return df

train_set = process_age(train_set,cut_points,label_names,'AGE')
test_set = process_age(test_set,cut_points,label_names,'AGE')

age_cat_pivot = train_set.pivot_table(index="AGE_categories",values="DEPRESSED")
age_cat_pivot.plot.bar()
plt.show()


: 

In [ ]:
#Checking on how much each attribute correlates with depression
corr_matrix = train_set.corr()
corr_matrix["DEPRESSED"].sort_values(ascending=False)

: 

In [ ]:
test_sets = test_set.replace('Male', 0).replace('Female', 1).replace('Student', 1).replace('Employeed', 2).replace('Unemployeed', 3).replace('Self-employeed', 4)
train_sets = train_set.replace('Male', 0).replace('Female', 1).replace('Student', 1).replace('Employeed', 2).replace('Unemployeed', 3).replace('Self-employeed', 4)

: 

In [ ]:
train_data =train_sets.fillna(0)
test_data = test_sets.fillna(0)


: 

In [ ]:
from sklearn.model_selection import train_test_split

# columns = ['GENDER', 'SAVINGS', 'DEBT', 'SALARY_SUSTAIN', 'ENJOY_JOB', 'BORROW_MONEY',
#            'FAMILY_DEPRESSED', 'MARITAL_STATUS', 'AGE', 'OCCUPATION', 'CHILDREN',
#            'HOUSEHOLD_SIZE', 'NO_CARE', 'BE_ALONE', 'FAILURE_FEELING', 'FAMILY_DEPRESSED']
columns = ['GENDER', 'MARITAL_STATUS', 'LOST_WEIGHT', ' 2_WEEK_SAD', 'TROUBLE _SLEEPING', 'TAKE_DRUGS', 'SUICIDE_THOUGHT',
               'FAILURE_DECISIONS', 'DEBT', 'DEPRESSED']
all_X = train_data[columns]
all_y = train_data['DEPRESSED']

train_X, test_X, train_y, test_y = train_test_split(all_X, all_y, test_size=0.2, random_state=0)
train_X.shape





: 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(train_X, train_y)


: 

In [ ]:
# #Prediction

# accuracy = accuracy_score(test_y, predictions)
# accuracy

# Actual class predictions
# rf_predictions = model.predict(test_y)
# # Probabilities for each class
# rf_probs = model.predict_proba(test_y)[:, 1]

# Actual class predictions
rf_predictions = model.predict(test_X)

# Probabilities for each class
rf_probs = model.predict_proba(test_X)[:, 1]



: 

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, rf_predictions))

: 

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(test_y, rf_predictions)
pd.DataFrame(conf_matrix, columns=['Depressed', 'Not_depressed'], index=[['Depressed', 'Not_depressed']])

: 

In [ ]:
import joblib

# Save the model to a file
joblib.dump(model, 'random_forest_model.pkl')


: 